In [298]:
import scrapy
import requests
from scrapy.http import TextResponse # xpath 연습

In [299]:
from fake_useragent import UserAgent
# 웹 페이지 연결
headers = {"User-Agent" : UserAgent().chrome}
response = requests.get("https://search.musinsa.com/ranking/best?u_cat_cd=002", headers = headers)
# TextResponse 객체 생성
text_response = TextResponse(response.url, body=response.text, encoding="utf-8")

In [300]:
text_response.xpath('//*[@id="goodsRankList"]/li/div[3]/div[2]/p[1]/a/@href').extract()

['https://www.musinsa.com/brands/beslow',
 'https://www.musinsa.com/brands/stoneisland',
 'https://www.musinsa.com/brands/musinsastandard',
 'https://www.musinsa.com/brands/thenorthface',
 'https://www.musinsa.com/brands/thenorthface',
 'https://www.musinsa.com/brands/nationalgeographic',
 'https://www.musinsa.com/brands/discoveryexpedition',
 'https://www.musinsa.com/brands/coor',
 'https://www.musinsa.com/brands/danswer',
 'https://www.musinsa.com/brands/spao',
 'https://www.musinsa.com/brands/thenorthface',
 'https://www.musinsa.com/brands/thenorthface',
 'https://www.musinsa.com/brands/nationalgeographic',
 'https://www.musinsa.com/brands/partimento',
 'https://www.musinsa.com/brands/diafvine',
 'https://www.musinsa.com/brands/nationalgeographic',
 'https://www.musinsa.com/brands/toffee',
 'https://www.musinsa.com/brands/insilence',
 'https://www.musinsa.com/brands/insilence',
 'https://www.musinsa.com/brands/calvinkleinjeans',
 'https://www.musinsa.com/brands/doffjason',
 'https:/

In [301]:
!scrapy startproject musinsa_crawler

Error: scrapy.cfg already exists in C:\Users\td708\notebook\musinsa_crawler


In [302]:
responser = requests.get("https://search.musinsa.com/ranking/best?u_cat_cd=002")
t_response = TextResponse(response.url, body=response.text, encoding="utf-8")

In [303]:
links_1 = t_response.xpath('//*[@id="goodsRankList"]/li/div[3]/div[2]/p/a/@href').extract()
links = links_1[1::2]

In [304]:
len(links)

90

In [305]:
link = links[1]
item_code = link.split("/")
item_code = item_code[5].split("?")
item_code = item_code[0]
item_code

'2064478'

In [306]:
res = requests.get(link)
t_res = TextResponse(res.url, body=res.text, encoding="utf-8")

In [307]:
# 브랜드
brand = t_response.xpath('//*[@id="goodsRankList"]/li[1]/div[3]/div[2]/p[2]/a/text()')[0].extract()
print(brand)

라퍼지스토어


In [308]:
# 첫 번째 링크를 이용해서 나머지 크롤링
link = links[1]

res = requests.get(link)
t_res = TextResponse(res.url, body=res.text, encoding="utf-8")

# 상품 id
item_id = link.split("/")
item_id = item_id[5].split("?")
item_id = item_id[0]
print(item_id)

# 상품명
title = t_response.xpath('//*[@id="goodsRankList"]/li[1]/div[3]/div[2]/p[3]/a/text()')[0].extract().strip()
print(title)

# 브랜드
brand = t_response.xpath('//*[@id="goodsRankList"]/li[1]/div[3]/div[2]/p[2]/a/text()')[0].extract()
print(brand)

# 원가
o_price = t_response.xpath('//*[@id="goodsRankList"]/li[1]/div[3]/div[2]/p[4]/del/text()')[0].extract()
o_price = o_price.replace(",","")
o_price = o_price.replace("원","")
print(o_price)

# 할인가
s_price = t_response.xpath('//*[@id="goodsRankList"]/li[1]/div[3]/div[2]/p[4]/text()')[-1].extract().strip()
s_price = s_price.replace(",", "")
s_price = s_price.replace("원", "")
print(s_price)

# 할인율 - (할인가 / 원가) * 100
discount_rate = round((1 - (int(s_price) / int(o_price))) * 100, 2)
discount_rate = "{}%".format(discount_rate)
print(discount_rate)

2064478
울리치 발마칸 오버 코트_Black
라퍼지스토어
173000
108000
37.57%


In [309]:
%%writefile musinsa_crawler/musinsa_crawler/items.py
import scrapy


class CrawlerItem(scrapy.Item):
    item_id = scrapy.Field()
    title = scrapy.Field()
    o_price= scrapy.Field()
    s_price = scrapy.Field()
    discount_rate = scrapy.Field()



Overwriting musinsa_crawler/musinsa_crawler/items.py


In [310]:
%%writefile musinsa_crawler/musinsa_crawler/spiders/spider.py
import scrapy
from musinsa_crawler.items import CrawlerItem

class Spider(scrapy.Spider):
    ### 바뀌면 안되는 변수들 ###
    name = "MusinsaRankingShop" # 프로그램 이름
    allow_domain = ["https://store.musinsa.com/app/"] # 허용할 도메인
    start_urls = ["https://search.musinsa.com/ranking/best?u_cat_cd=002"]
    #################################
    
    # 바뀌면 안되는 함수
    def parse(self, response):
        # 상세 페이지 링크 목록 얻어오기
        links_1 = response.xpath('//*[@id="goodsRankList"]/li/div[3]/div[2]/p/a/@href').extract()
        links = links_1[1::2]
        
        # 페이지 하나하나 들어가면서 페이지 크롤링
        for link in links:
            # callback : 요청 완료 후 parsing 된 내용을 처리할 함수
            yield scrapy.Request(link, callback=self.page_content)
    
    # 메소드 이름은 바꿔도 상관 없으나 매개변수는 바꾸지 말 것
    def page_content(self, response): # response : 상세 페이지 결과
        item = CrawlerItem()
        
        # 상품 id
        item_id = link.split("/")
        item_id = item_id[5].split("?")
        item_id = item_id[0]
        

        # 상품명
        title = response.xpath('//*[@id="goodsRankList"]/li[1]/div[3]/div[2]/p[3]/a/text()')[0].extract().strip()
        

        # 브랜드
        brand = response.xpath('//*[@id="goodsRankList"]/li[1]/div[3]/div[2]/p[2]/a/text()')[0].extract()
        



        # 할인가
        s_price = response.xpath('//*[@id="goodsRankList"]/li[1]/div[3]/div[2]/p[4]/text()')[-1].extract().strip()
        s_price = s_price.replace(",", "")
        s_price = s_price.replace("원", "")
        
        
        # 원가가 없고 할인가만 있는 경우가 있으면 원가에 할인가 넣어주기
        try: 

            # 원가
            o_price = response.xpath('//*[@id="goodsRankList"]/li[1]/div[3]/div[2]/p[4]/del/text()')[0].extract()
            o_price = o_price.replace(",","")
            o_price = o_price.replace("원","")

            
   
        except:
        
            
            item["o_price"] = s_price
            
                # 할인율 - (1 - (할인가 / 원가) * 100)
        discount_rate = round((1 - (int(item["s_price"]) / int(item["o_price"]))) * 100, 2)
        discount_rate = "{}%".format(discount_rate)

        item["discount_rate"] = discount_rate
        
        
        yield item

Overwriting musinsa_crawler/musinsa_crawler/spiders/spider.py


In [311]:
%%writefile run.sh

cd musinsa_crawler
scrapy crawl MusinsaRankingShop

Overwriting run.sh


In [312]:
!chmod +x run.sh

'chmod'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [313]:
!./run.sh

'.'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [314]:
import pandas as pd
df_mbest = pd.read_csv("./musinsa_crawler/MusinsaRankingShop.csv", encoding="utf-8")
df_mbest.head()

FileNotFoundError: [Errno 2] No such file or directory: './musinsa_crawler/MusinsaRankingShop.csv'